# Aulão LLM's e Agentes - Básico de LangChain

## Etapa 1 - Pergunta de Engajamento


Construir um sistema simples que responde a perguntas sobre um tópico específico da escola.

Isso é bem simples, mas é legal para mostrar como o LangChain organiza, estrutura e facilita o uso de LLMs com prompts reutilizáveis.

### 1) Sem o Langchain

In [1]:
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()
client = OpenAI()

# Inputs
segmento = "Ensino Médio"

# Prompt montado na mão - vê que aqui eu usei uma f-string 
prompt = f"""Você é um assistente de admissões de uma escola de excelência.
O pai está interessado no {segmento}.
Gere 3 opções de perguntas iniciais para entender a principal preocupação dele (ex: vestibular, segurança, socialização)."""

# com o gpt seria assim(apesar de ter várias outras formas):
response = client.responses.create(
    model="gpt-5-nano",
    input=prompt
)

print(response.output_text)

Aqui vão 3 opções de perguntas iniciais, cada uma focando em uma possible principal preocupação do pai:

- Pergunta 1 (vestibular/universidade): “Quais são as suas principais preocupações em relação à preparação do seu filho para o vestibular e para a universidade? há metas ou prazos específicos que você espera que a escola ajude a alcançar?”

- Pergunta 2 (segurança): “Quais aspectos de segurança, ambiente e bem-estar são mais importantes para você ao escolher o ensino médio para o seu filho?”

- Pergunta 3 (socialização/desenvolvimento): “Quão importante é para você o aspecto social e o desenvolvimento socioemocional (convivência, atividades extracurriculares, mentoria) no ensino médio?”


Ou com o Gemini

In [2]:
from google import genai

client = genai.Client()

# Inputs
segmento = "Ensino Médio"

# Prompt montado na mão - vê que aqui eu usei uma f-string 
prompt = f"""Você é um assistente de admissões de uma escola de excelência.
O pai está interessado no {segmento}.
Gere 3 opções de perguntas iniciais para entender a principal preocupação dele (ex: vestibular, segurança, socialização)."""

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt,
)

print(response.text)

Olá! É um prazer recebê-lo(a) e discutir o futuro educacional do seu filho(a) aqui na nossa escola. Para que eu possa oferecer as informações mais relevantes e personalizadas, gostaria de fazer algumas perguntas iniciais para entender o que é mais importante para você e sua família neste momento.

Aqui estão 3 opções de perguntas que poderíamos usar:

1.  **Foco no Objetivo Geral e no Futuro:**
    "Bem-vindo(a)! Para começarmos, adoraria entender o que os trouxe até nós hoje. Pensando no Ensino Médio, qual é o principal objetivo ou a expectativa mais importante que você tem para o seu filho(a) nesta fase, seja academicamente, socialmente ou em relação ao futuro?"

2.  **Foco nas Prioridades de Escolha da Escola:**
    "É uma grande decisão escolher a escola certa para o Ensino Médio. Ao considerar as opções, quais são os aspectos que mais pesam na sua decisão por uma escola? Existe alguma área específica – seja a preparação para a universidade, o desenvolvimento pessoal, a segurança o

### 2) Com o Langchain

In [3]:
from langchain_openai import ChatOpenAI
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import PromptTemplate

# Inputs
segmento = "Ensino Médio"

# usar um template é mais bizu
template = PromptTemplate(
    template=""" Você é um assistente de admissões de uma escola de excelência.
    O pai está interessado no {segmento}.
    Gere 3 opções de perguntas iniciais para entender a principal preocupação dele (ex: vestibular, segurança, socialização).
    """,
    input_variables=["segmento"], # colocamos as variáveis de input
)

prompt = template.format(
    segmento=segmento,
)

# gpt
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)
# gemini
#llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash", temperature=0.7)
# esse invoke é bem útil, você pode usar tanto o "prompt" quanto um histórico de mensagens, etc
resposta = llm.invoke(prompt)
print(resposta.content)

C:\Users\fealp\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Claro! Aqui estão três opções de perguntas iniciais que podem ajudar a entender as principais preocupações do pai em relação ao Ensino Médio:

1. **Qual é a sua principal preocupação em relação à preparação do seu filho para o vestibular e o futuro acadêmico?**
   
2. **Como você avalia a importância da segurança e do ambiente escolar para o bem-estar do seu filho durante o Ensino Médio?**
   
3. **Quais são suas expectativas em relação à socialização e ao desenvolvimento de habilidades interpessoais do seu filho nesse período de transição?**

Essas perguntas podem ajudar a identificar as prioridades e preocupações do pai, permitindo uma conversa mais direcionada sobre como a escola pode atender às suas necessidades.


Aqui vemos a 1º facilidade do langchain, a versatilidade de usar diferentes LLM's;
Vê como é complicado trocar o modelo de maneira rápida e dessa forma a gente só muda o parâmetro `llm` e a saída é a mesma.

Mas temos outra utilidade, e se quisersemos obter uma saída estruturada, como um JSON?

Aqui vamos ver um exemplo básico primeiro, usando a saída como string mesmo para vermos como funciona o parser e a cadeia.

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser


template = PromptTemplate(
    template="""
    Atue como um assistente de admissões da escola 'Futuro Brilhante'.
    
    Sua tarefa é escrever uma mensagem curta de 'quebra-gelo' para enviar no WhatsApp
    para um responsável interessado no segmento: {segmento}.
    
    Contexto da família:
    - O foco principal deles parece ser: {foco_familia}.
    - O tom da conversa deve ser: {tom_de_voz}.
    
    A mensagem deve terminar com uma pergunta aberta para engajar o responsável.
    """,
    input_variables=["segmento", "foco_familia", "tom_de_voz"]
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Construção da Cadeia - template + llm + parser de saída(aqui só string mesmo)
cadeia = template | llm | StrOutputParser()

# aqui o invoke a gente pode só colocar as input variables e usar o template
resposta = cadeia.invoke(
    {
        "segmento": "Ensino Médio",
        "foco_familia": "aprovação em universidades federais",
        "tom_de_voz": "confiante e profissional"
    }
)

print(resposta)

Olá! 👋 Sou o assistente de admissões da escola Futuro Brilhante. Fico feliz em saber que você está considerando nosso Ensino Médio para seu filho(a)! Aqui, temos um forte compromisso com a preparação para as universidades federais, oferecendo um acompanhamento personalizado e recursos que fazem a diferença. 

Quais são as principais expectativas que você tem em relação à formação do seu filho(a) nesse período tão importante?


## Etapa 2 - Chains com OutputParser
Agora quero tornar o analisador de atendimento da Etapa 1 estruturado e confiável. Queremos que a saída da LLM não seja apenas um texto solto, mas sim um JSON com os seguintes campos:

- `resumo_pedido`: um resumo do que o responsável deseja
- `dados_contato`: um dicionário com os dados identificados (Nome, Telefone, Email)
- `acao_recomendada`: a próxima ação sugerida para o atendente

In [5]:
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from pprint import pprint
from typing import Optional

# schema da resposta que definimos do Pydantic
class AnaliseAtendimento(BaseModel):
    resumo_pedido: str = Field(description="Resumo da solicitação do responsável")
    dados_contato: Optional[dict] = Field(default=None, description="Dicionário com chaves Nome, Contatos") # pode ser nulo
    acao_recomendada: str = Field(description="Ação sugerida: Agendar Visita, Enviar PDF ou Ligar")

parser = JsonOutputParser(pydantic_object=AnaliseAtendimento)

template = PromptTemplate(
    template="""Analise a mensagem de atendimento recebida pelo canal "{canal}", referente ao segmento "{segmento}".
    
    Mensagem do Cliente: "{mensagem}"
    
    Extraia as informações e inclua:
    - Um resumo do pedido
    - Um dicionário com dados de contato
    - A ação recomendada

    {format_instructions}
""",
    input_variables=["canal", "segmento", "mensagem"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Cadeia: template + llm + parser(aqui o parser é o JsonOutputParser, que vai gerar o JSON estruturado)
cadeia = template | llm | parser

resposta = cadeia.invoke({
    "canal": "WhatsApp",
    "segmento": "Ensino Fundamental",
    "mensagem": "Oi, sou a Ana. Meu filho Pedro está no 4º ano. Gostaria de saber os valores e se tem vaga de manhã. Meu zap é esse mesmo."
})

pprint(resposta)

{'acao_recomendada': 'Enviar informações sobre valores e disponibilidade de '
                     'vaga para o 4º ano pela WhatsApp.',
 'dados_contato': {'Contatos': {'WhatsApp': 'sim'}, 'Nome': 'Ana'},
 'resumo_pedido': 'A cliente Ana solicita informações sobre valores e '
                  'disponibilidade de vagas para seu filho Pedro, que está no '
                  '4º ano, no período da manhã.'}


Perceba que definimos os dados de contato como um dicionário de maneira aberta (bem "ruim"), mas poderia ser uma classe Pydantic também:

In [6]:
class DadosContato(BaseModel):
    nome: Optional[str] = Field(default=None, description="Nome do responsável")
    contatos: Optional[dict] = Field(default=None, description="Informações de contato (telefone, email, etc.)")

class AnaliseAtendimento(BaseModel):
    resumo_pedido: str = Field(description="Resumo da solicitação do responsável")
    dados_contato: DadosContato = Field(description="Dicionário com chaves Nome, Telefone e Email (se encontrados)")
    acao_recomendada: str = Field(description="Ação sugerida: Agendar Visita, Enviar PDF ou Ligar")

parser = JsonOutputParser(pydantic_object=AnaliseAtendimento)

template = PromptTemplate(
    template="""Analise a mensagem de atendimento recebida pelo canal "{canal}", referente ao segmento "{segmento}".
    
    Mensagem do Cliente: "{mensagem}"
    
    Extraia as informações e inclua:
    - Um resumo do pedido
    - Um dicionário com dados de contato
    - A ação recomendada

    {format_instructions}
""",
    input_variables=["canal", "segmento", "mensagem"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7)

# Cadeia: template + llm + parser(aqui o parser é o JsonOutputParser, que vai gerar o JSON estruturado)
cadeia = template | llm | parser

resposta = cadeia.invoke({
    "canal": "WhatsApp",
    "segmento": "Ensino Fundamental",
    "mensagem": "Oi, sou a Ana. Meu filho Pedro está no 4º ano. Gostaria de saber os valores e se tem vaga de manhã. Meu zap é esse mesmo."
})

pprint(resposta)

{'acao_recomendada': 'Enviar informações sobre valores e disponibilidade de '
                     'vagas para o 4º ano de manhã via WhatsApp.',
 'dados_contato': {'contatos': {'whatsapp': 'sim'}, 'nome': 'Ana'},
 'resumo_pedido': 'A cliente Ana deseja informações sobre valores e '
                  'disponibilidade de vagas para o 4º ano de seu filho Pedro '
                  'no período da manhã.'}


Com esse exemplo que passamos, você já pode perceber a vantagem disso, bem além de você enviar isso para um banco de dados, você pode usar isso para alimentar outras cadeias, agentes, etc.

## Etapa 3 — Encadeamento de múltiplas chains

Vamos criar um sistema de "Reengajamento Automatizado" que analisa conversas pausadas, seleciona conteúdo relevante e gera mensagens de retomada personalizadas.
Fluxo:

1. Analisa o histórico e identifica em que fase o lead parou
2. Cruza o diagnóstico com banco de materiais e escolhe o conteúdo adequado
3. Gera texto de retomada contextualizado de maneira personalizada

Para isso, vamos usar o arquivo `marketings.json`

Primeiramente, vamos pensar na classificação do lead, para depois irmos para a seleção de conteúdo e geração da mensagem.

In [9]:
from typing import Literal, List
import json
load_dotenv()

TipoPerfil = Literal["Objetivo", "Detalhista", "Afetivo", "Desconfiado", "Negociador"]

# à título de exemplo, mostrando como usar um "dropdown" de possíveis classificações para perfil_comunicacao - usar o Literal
# vai ser mais útil nas tags
class Diagnostico(BaseModel):
    """Diagnóstico de interação com responsável."""
    resumo_conversa: str = Field(description="Resumo dos principais pontos discutidos na conversa")
    perfil_comunicacao: TipoPerfil = Field(description="Estilo de comunicação predominante identificado no responsável")
    hipotese_inatividade: str = Field(description="Principal motivo provável para a falta de resposta ou silêncio")
    foco_interesse: str = Field(description="Tema ou aspecto que mais despertou interesse")


# Configuração do LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)

# Parsers
parser_diag = JsonOutputParser(pydantic_object=Diagnostico)

# Prompt
# vamos colocar o histórico no prompt, mas vamos ver futuramente como melhorar essa abordagem
prompt_diag = PromptTemplate(
    template="""Você é um analista de vendas educacionais especializado em diagnóstico de leads.
Sua tarefa é analisar o histórico de conversa abaixo e extrair insights sobre o responsável.
Seja preciso, objetivo e baseie-se apenas nas informações fornecidas no histórico.

histórico:
{historico}
{format_instructions}""",
    input_variables=["historico"],
    partial_variables={"format_instructions": parser_diag.get_format_instructions()}
)


chain_diag = prompt_diag | llm | parser_diag


historico = [
    {"role": "assistant", "content": "Olá! Bem-vindo à Escola Futuro. Como posso ajudar?"},
    {"role": "human", "content": "Oi, queria saber como funciona o ensino fundamental 1."},
    {"role": "assistant", "content": "Claro! Focamos em autonomia e projetos. Quer agendar uma visita?"},
    {"role": "human", "content": "Ah, legal. Mas vocês usam apostilas ou livros?"},
    {"role": "assistant", "content": "Usamos livros didáticos e projetos maker."},
    {"role": "human", "content": "Entendi. Vou ver com a minha esposa."},
    {"role": "system", "content": "Lead inativo há 3 dias."}
]

resultado = chain_diag.invoke({
    "historico": json.dumps(historico, ensure_ascii=False)
})

pprint(resultado)

{'foco_interesse': 'Métodos de ensino utilizados, especificamente sobre '
                   'apostilas e livros didáticos.',
 'hipotese_inatividade': 'O responsável pode estar aguardando a opinião da '
                         'esposa antes de prosseguir com a decisão.',
 'perfil_comunicacao': 'Objetivo',
 'resumo_conversa': 'O responsável demonstrou interesse em saber como funciona '
                    'o ensino fundamental 1, questionou sobre o uso de '
                    'apostilas ou livros e mencionou que iria discutir com a '
                    'esposa antes de tomar uma decisão.'}


Ok. Vamos passar para a parte da busca.

Aqui vamos passar os conteúdos de marketing para o prompt basicamente por didática, obviamente isso não escala, mas você poderia fazer várias outras abordagens, vou te mostrar 2:

1. Na realização do diagnóstico, tente classificar o Lead em tags. Essas tags podem ser usadas para buscar conteúdos relacionados que tem as mesmas tags no banco de dados, então você faz um ranking e escolhe.

2. Vamos futuramente ver RAG (Retrieval-augmented generation), onde você pode indexar esses conteúdos em um vetor e fazer uma busca vetorial para trazer os conteúdos mais relevantes. Você pode fazer uma busca por similaridade com o diagnóstico/resumo e trazer os conteúdos mais similares. Em palavras simples, você vai conseguir trazer os conteúdos que tem descrições mais similares à descrição/resumo do lead. Essa é a abordagem mais robusta e escalável se você tem muitos conteúdos que são similares entre si(Ex: Pela abordagem 1, tanto faz você escolher um conteúdo que tem as tags "Segurança" e "Infraestrutura" e outro que também tem as mesmas tags).


In [10]:
import json
from typing import Literal
from pydantic import BaseModel, Field

class DecisaoConteudo(BaseModel):
    id_material: str = Field(description="O ID do material escolhido da lista.")
    titulo_material: str = Field(description="O título do material escolhido.")
    match_reasoning: str = Field(description="Explicação breve do motivo deste conteúdo é adequado para este perfil de responsável.")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
parser = JsonOutputParser(pydantic_object=DecisaoConteudo)

template = PromptTemplate(
    template="""
    Atue como um Especialista de Marketing Escolar.
    
    Sua missão é escolher o MELHOR material de apoio para enviar a um lead, baseando-se no perfil dele.
    
    Perfil do Lead:
    - Estilo de Comunicação: {perfil_comunicacao}
    - Foco de Interesse: {foco_interesse}
    - Hipotese para Inatividade: {hipotese_inatividade}

    Conteúdos disponíveis:
    {banco_conteudos}
    
    Regras:
    1. Escolha apenas 1 material que tenha o maior "Match" com o interesse do lead.
    2. Retorne o ID exato.
    
    {format_instructions}
    """,
    input_variables=["perfil_comunicacao", "foco_interesse", "hipotese_inatividade", "banco_conteudos"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = template | llm | parser


# Exemplo: Vamos colocar obtidos anteriormente, por exemplo     
perfil_comunicacao = "Objetivo"
foco_interesse = "Métodos de ensino e materiais utilizados (livros didáticos e projetos maker)."
hipotese_inatividade = "O responsável pode estar aguardando a opinião da esposa antes de tomar uma decisão."

# Carregando o banco de conteúdos de marketing
banco_conteudos = json.load(open("marketings.json", "r", encoding="utf-8"))
# passando para string
# nesse caso passamos tudo que estava no banco, mas você poderia passar somente as descrições e os IDs
banco_conteudos = json.dumps(banco_conteudos, ensure_ascii=False)

resultado = chain.invoke({
    "perfil_comunicacao": perfil_comunicacao,
    "foco_interesse": foco_interesse,
    "hipotese_inatividade": hipotese_inatividade,
    "banco_conteudos": banco_conteudos
})

print(json.dumps(resultado, indent=2, ensure_ascii=False))

{
  "id_material": "MAT_001",
  "titulo_material": "Comparativo: Método Tradicional vs Construtivista",
  "match_reasoning": "Este material aborda diretamente os métodos de ensino, que é o foco de interesse do lead, permitindo uma análise objetiva das abordagens pedagógicas da escola."
}


Massa! Vamos ver agora a parte da mensagem final.

In [11]:
class MensagemResgate(BaseModel):
    texto: str = Field(description="Texto da mensagem de resgate para o WhatsApp")

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.7) # vamos colocar uma temperatura maior para ser mais criativo e natural
parser = JsonOutputParser(pydantic_object=MensagemResgate)


template = PromptTemplate(
    template="""
    Atue como um Resgatador de Lead por meio de uma mensagem de WhatsApp.
    Escreva uma mensagem de retomada de contato juntamente com o conteúdo de marketing disponibilizado.
    O conteúdo vai ser enviado junto com a mensagem, então não repita informações do material.

    Conteúdo:
    - Título: {titulo_material}
    - Motivo para a escolha do conteúdo: {match_reasoning}
    
    {format_instructions}
    """,
    input_variables=["perfil", "interesse", "motivo_pausa", "titulo_material", "match_reasoning"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

chain = template | llm | parser

# vamos supor que os resultados da chain anterior foram:
titulo_material = "Comparativo: Método Tradicional vs Construtivista"
match_reasoning = "Este material aborda diretamente os métodos de ensino, que é o foco de interesse do lead, proporcionando uma comparação clara entre abordagens pedagógicas."

mensagem = chain.invoke({
    "titulo_material": titulo_material,
    "match_reasoning": match_reasoning
})

print(mensagem)

{'texto': 'Olá! Espero que você esteja bem. Estou retomando o contato para compartilhar um material que acredito ser do seu interesse: um comparativo entre o Método Tradicional e o Construtivista. Esse conteúdo oferece uma visão clara sobre as diferentes abordagens pedagógicas, que podem ser muito úteis para suas decisões. Se precisar de mais informações ou quiser discutir o assunto, estou à disposição!'}


Beleza, mas como eu faço esse processo todo de maneira encadeada? Ou seja, quero que a saída de uma etapa seja a entrada da próxima. Vou reescrever o código todo aqui para ficar mais claro:

In [12]:
# para ver o processo ocorrendo por baixo dos panos, vamos colocar o set_debug

from langchain_core.globals import set_debug
set_debug(True)

# Carregando o banco de conteúdos de marketing
banco_conteudos = json.load(open("marketings.json", "r", encoding="utf-8"))
# passando para string
# nesse caso passamos tudo que estava no banco, mas você poderia passar somente as descrições e os IDs
banco_conteudos = json.dumps(banco_conteudos, ensure_ascii=False)

TipoPerfil = Literal["Objetivo", "Detalhista", "Afetivo", "Desconfiado", "Negociador"]

class Diagnostico(BaseModel):
    """Diagnóstico de interação com responsável."""
    resumo_conversa: str = Field(description="Resumo dos principais pontos discutidos na conversa")
    perfil_comunicacao: TipoPerfil = Field(description="Estilo de comunicação predominante identificado no responsável")
    hipotese_inatividade: str = Field(description="Principal motivo provável para a falta de resposta ou silêncio")
    foco_interesse: str = Field(description="Tema ou aspecto que mais despertou interesse")

class DecisaoConteudo(BaseModel):
    id_material: str = Field(description="O ID do material escolhido da lista.")
    titulo_material: str = Field(description="O título do material escolhido.")
    match_reasoning: str = Field(description="Explicação breve do motivo deste conteúdo é adequado para este perfil de responsável.")

class MensagemResgate(BaseModel):
    texto: str = Field(description="Texto da mensagem de resgate para o WhatsApp")

# llm's
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.2)
llm_resgate = ChatOpenAI(model="gpt-4o-mini", temperature=0.7) # mais criativo para resgate

# parsers
parser_diag = JsonOutputParser(pydantic_object=Diagnostico)
parser_conteudo = JsonOutputParser(pydantic_object=DecisaoConteudo)
parser_resgate = JsonOutputParser(pydantic_object=MensagemResgate)

# templates
template_diag = PromptTemplate(
    template="""Você é um analista de vendas educacionais especializado em diagnóstico de leads.
Sua tarefa é analisar o histórico de conversa abaixo e extrair insights sobre o responsável.
Seja preciso, objetivo e baseie-se apenas nas informações fornecidas no histórico.

histórico:
{historico}
{format_instructions}""",
    input_variables=["historico"],
    partial_variables={"format_instructions": parser_diag.get_format_instructions()}
)

template_conteudo = PromptTemplate(
    template="""
    Atue como um Especialista de Marketing Escolar.
    
    Sua missão é escolher o MELHOR material de apoio para enviar a um lead, baseando-se no perfil dele.
    
    Perfil do Lead:
    - Estilo de Comunicação: {perfil_comunicacao}
    - Foco de Interesse: {foco_interesse}
    - Hipotese para Inatividade: {hipotese_inatividade}

    Conteúdos disponíveis:
    {banco_conteudos}
    
    Regras:
    1. Escolha apenas 1 material que tenha o maior \"Match\" com o interesse do lead.
    2. Retorne o ID exato.
    
    {format_instructions}
    """,
    input_variables=["perfil_comunicacao", "foco_interesse", "hipotese_inatividade", "banco_conteudos"],
    partial_variables={
        "banco_conteudos": banco_conteudos,
        "format_instructions": parser_conteudo.get_format_instructions()
    }
)
template_resgate = PromptTemplate(
    template="""
    Atue como um Resgatador de Lead por meio de uma mensagem de WhatsApp.
    Escreva uma mensagem de retomada de contato juntamente com o conteúdo de marketing disponibilizado.
    O conteúdo vai ser enviado junto com a mensagem, então não repita informações do material.

    Conteúdo:
    - Título: {titulo_material}
    - Motivo para a escolha do conteúdo: {match_reasoning}
    
    {format_instructions}
    """,
    input_variables=["perfil_comunicacao", "foco_interesse", "hipotese_inatividade", "titulo_material", "match_reasoning"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

# chains
chain_diag = template_diag | llm | parser_diag
chain_conteudo = template_conteudo | llm | parser_conteudo
chain_resgate = template_resgate | llm_resgate | parser_resgate
# principal agora;
chain_principal = chain_diag | chain_conteudo | chain_resgate

historico = [
    {"role": "assistant", "content": "Olá! Bem-vindo à Escola Futuro. Como posso ajudar?"},
    {"role": "human", "content": "Oi, queria saber como funciona o ensino fundamental 1."},
    {"role": "assistant", "content": "Claro! Focamos em autonomia e projetos. Quer agendar uma visita?"},
    {"role": "human", "content": "Ah, legal. Mas vocês usam apostilas ou livros?"},
    {"role": "assistant", "content": "Usamos livros didáticos e projetos maker."},
    {"role": "human", "content": "Entendi. Vou ver com a minha esposa."},
    {"role": "system", "content": "Lead inativo há 3 dias."}
]

mensagem = chain_principal.invoke({
    "historico": historico,
})



[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "historico": [
    {
      "role": "assistant",
      "content": "Olá! Bem-vindo à Escola Futuro. Como posso ajudar?"
    },
    {
      "role": "human",
      "content": "Oi, queria saber como funciona o ensino fundamental 1."
    },
    {
      "role": "assistant",
      "content": "Claro! Focamos em autonomia e projetos. Quer agendar uma visita?"
    },
    {
      "role": "human",
      "content": "Ah, legal. Mas vocês usam apostilas ou livros?"
    },
    {
      "role": "assistant",
      "content": "Usamos livros didáticos e projetos maker."
    },
    {
      "role": "human",
      "content": "Entendi. Vou ver com a minha esposa."
    },
    {
      "role": "system",
      "content": "Lead inativo há 3 dias."
    }
  ]
}
[chain/start] [chain:RunnableSequence > prompt:PromptTemplate] Entering Prompt run with input:
{
  "historico": [
    {
      "role": "assistant",
      "content": "Olá! Bem-vindo à Escol

`Output Final`:

_Olá! Tudo bem? Espero que você esteja tendo um ótimo dia! Eu gostaria de retomar nosso contato e compartilhar um material interessante que aborda o comparativo entre o Método Tradicional e o Construtivista. Acredito que você achará útil para entender melhor as abordagens pedagógicas que estamos discutindo. Estou à disposição para conversar mais sobre o assunto!_

_Por que não fazer de maneira unificada? Sla, um único prompt que faça tudo?_ 

Desse jeito você consegue trabalhar de maneira mais individual em cada etapa, testar, validar, ajustar, etc. O que é mais valioso é que você consegue separar as etapas de pensamento da IA e não fazer com que ela pense do jeito dela. Você tem bem mais controle de tudo que tá acontecendo. Além disso, você pode reutilizar essas cadeias em outros fluxos também. 

**Observação**: no pipe puro `A | B | C`, você pode "perder" campos do estado(as variáveis), que evidentemente podem ser importantes.
Aqui a gente usa `RunnablePassthrough.assign(...)` para **preservar** diagnóstico + decisão de conteúdo até o final.

In [15]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
# vou colocar false porque já sugou
set_debug(False)

# podiamos melhorar isso assim:
chain_principal = (
    # diagnóstico (gera: resumo_conversa, perfil_comunicacao, hipotese_inatividade, foco_interesse)
    RunnablePassthrough.assign(diagnostico=chain_diag)
    # “espalha” os campos do diagnóstico no topo (pra alimentar o próximo prompt)
    | RunnableLambda(lambda x: {**x, **x["diagnostico"]})
    # escolhe conteúdo (gera: id_material, titulo_material, match_reasoning)
    | RunnablePassthrough.assign(conteudo=chain_conteudo)
    # espalha campos do conteúdo no topo (pra alimentar resgate)
    | RunnableLambda(lambda x: {**x, **x["conteudo"]})
    # mensagem final
    | RunnablePassthrough.assign(mensagem_resgate=chain_resgate)
)

resposta = chain_principal.invoke({"historico": historico})

# O que interessa no final:

print(f'diagnóstico: \n\n {resposta["diagnostico"]}')
print(f'\nconteúdo escolhido: \n\n {resposta["conteudo"]}')
print(f'\nmensagem de resgate: \n\n {resposta["mensagem_resgate"]}')

diagnóstico: 

 {'resumo_conversa': 'O responsável demonstrou interesse em saber como funciona o ensino fundamental 1, questionou sobre o uso de apostilas ou livros e mencionou que iria discutir a decisão com a esposa.', 'perfil_comunicacao': 'Objetivo', 'hipotese_inatividade': 'O responsável está consultando a esposa antes de tomar uma decisão.', 'foco_interesse': 'Métodos de ensino e materiais utilizados (apostilas ou livros)'}

conteúdo escolhido: 

 {'id_material': 'MAT_001', 'titulo_material': 'Comparativo: Método Tradicional vs Construtivista', 'match_reasoning': 'Este material aborda diretamente os métodos de ensino, que é o foco de interesse do lead, permitindo uma análise objetiva entre diferentes abordagens pedagógicas.'}

mensagem de resgate: 

 {'texto': 'Olá! Espero que você esteja bem. Estou entrando em contato para ver como andam suas reflexões sobre métodos de ensino. Para te ajudar nessa análise, estou compartilhando um material que fizemos sobre o comparativo entre o 

Boa! Finalizamos essa parte inicial que dá pra fazer bastante coisa que enriquecem bastante o produto, seja fazendo uma automação para resumir informações do Lead, ferramentas de reengajamento, avaliação de reuniões de vendas.

Na aula que vem vamos ver como trabalhar com RAG e memória para enriquecer ainda mais nossos sistemas.